In [ ]:
import glob
from datetime import datetime
import os
from multiprocessing import Pool

import numpy as np
from scipy.ndimage import gaussian_filter

from uncertainty import ComputeSicUncertainty

%matplotlib inline

In [ ]:
n_steps = 2210  # should be 2210 to cover 6 years

sia_dir = 'data2/Anton/sia/cdr_1991_2023'
mesh_dir = f'{sia_dir}/mesh'
unc_dir = f'{sia_dir}/unc'
sid_dir = 'data2/Anton/sia_sid_cdr_postproc'
ifiles = sorted(glob.glob(f'{sid_dir}/*/ice_drift*npz'))
idates = [datetime.strptime(os.path.basename(ifile).split('-')[-1].split('.')[0], '%Y%m%d%H%M%S')
          for ifile in ifiles]

mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']
mask = np.load(mesh_init_file)['mask']
mask_f = gaussian_filter(mask.astype(float), 1, truncate=1)

In [ ]:
start_indices = [i for i,j in enumerate(idates) if j.year > 2018 and j.month == 9 and j.day == 5]
start_indices

In [ ]:
csu = ComputeSicUncertainty(ifiles, idates, n_steps, mesh_init_file, mesh_dir, unc_dir, xc, yc)
with Pool(2) as p:
    p.map(csu.compute_uncertainty, start_indices)


Uncertainty of the produced sea ice age variable is computed from the uncertainties of the ice age fractions ($C_i$) multiplied by age ($A$) of the fraction: 

$\sigma_{SIA}^2 = \sum{ A_i * \sigma_{C_{iY}}^2 }, i=1..6$,

where $i$ is the number of the sea ice age fraction: i=1 for one-year old ice, 2 for two-year old ice, etc.

As each ice age fraction is computed from values of MYI advected for various time, the uncertainty of each fraction is computed from uncertaimties of advected MYI fields:

$\sigma_{C_{iY}}^2 = \sum{ \sigma_{C_{A,j}}^2 }, j = 1..i$,

where $j$ is the duration of advection: 1 - below one year of advection, 2 - below two years of advection, etc. Therefore, $j\le i$.


NB: Not dureation but a 'stream'. REWRITE!



Uncertainty of the FYI is computed using the uncertainty from the satellite SIC observations and the youngest advected MYI field:

$\sigma_{FYI}^2 = \sigma_{C_{OBS}}^2 + \sigma_{C_{A1}}^2$

Uncertaimty of the advected MYI field is computed from uncertainties of the satellite derived SIC and SID:

$\sigma_{C_{A,j}}^2 = \sigma_{SIC,j}^2 + \sigma_{SID,j}^2$


Both SIC and SID uncertaimties are iteratively accumulated during the advection of the MYI fields. 

The concentration in the advected field is capped by the observed concentration. In other words it is a minimum values of the advected and the observed concentration (see Eq. NNN). Therefore, the uncertainty of the advected field is the uncertainty of the minimal concentration:

$\sigma_{SIC,j,n}^2 = $

$\left< \sigma_{SIC,j,n-1}^2 \right> \text{, if } SIC_{n-1} < C_{OBS,n}$

$\sigma_{C_{OBS,n}}^2 \text{, otherwise}$

where $n$ denotes the current step, $n-1$ - the previous step, $<>$ - operation of advection.


The uncertainty associated with the drift is computed the following way. First, the uncertainty of the smoothed ice drift product ($\sigma_{S}$) is computed using the provided uncertainties of the OSI-SAF SID product ($\sigma_D$):

$\sigma_{S}^2 = \frac{\sum_k^N{\sigma_{D_k}}}{N}$,

where $\sigma_{D_k}$ is the uncertaimty of ice drift in a $k$-th neighbour and $N$ - number of neighbours.

Next, the integrated uncertainty of the ice drift ($\sigma_{I}$) is iteratively acumulated with advection:

$\sigma_{I,n}^2 = \left< \sigma_{I,n-1}^2 \right> + \sigma_{S}^2$

After N steps of advection, the integrated uncertainty of sea ice drift defines a radius of a circle, where the advected mesh element could have drifted. This circle may include other elements with their respective concentrations. Therefore the uncertainty of concentration, associated with the uncertaimty of ice drift $\sigma_{SID,j}$ is computed as a standard deviation of concentrations in this circle:

$\sigma_{SID,j}^2 = (\sum_m^M{ C_{A,j,m} - \hat{C}_{A,j})}^2/M$

where $m$ is the index of the elements in the circle, $\hat{C}_{A,j}$ average concentration in the circle, M – number of elements in the circle.
